In [1]:
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt

# pynq Overlay
from pynq import Overlay

# audio Codec driver module
from pynq.lib.audio import AudioADAU1761

In [2]:
# overlay laden
ol = Overlay("audio_test_v2_HP_Filter.bit")
# Check IP names
ol.ip_dict.keys()

dict_keys(['audio_codec_ctrl_0', 'i2s_receiver_0', 'i2s_transmitter_0', 'processing_system7_0'])

### Notizen
Design funktioniert nicht <br>
.bypass funktioniert aber auch nicht -> PL wird also nicht umgangen <br>
Codec wird wahrscheinlich richtig init, fefehle Funktionieren, keine Fehlermeldungen <br>
-> buffer bleibt Leer -> Filter entfernt <br>
-> buffer voll 16777215 (Hex: 0xFFFFFF) -> Filter hat Funktioniert => Filter war nicht problem <br>
-> receiver und transmitetr **vor** dem codec_controller verschalten

### Pot Ursachen
I2S Transmitter / receiver falsch Konfig <br>
Clocksignale: AXI-Stream 50MHz aber I2S nur 10MHz <br>
codec_addr: irgendwas damit 

In [3]:
import os
for i in range(4):  # prüfe /dev/uio0 bis /dev/uio3
    try:
        with open(f"/sys/class/uio/uio{i}/name", "r") as f:
            name = f.read().strip()
            print(f"uio{i} → {name}")
    except FileNotFoundError:
        continue

uio0 → audio-codec-ctrl
uio1 → fabric


In [4]:
# Bezeichnung des Audio Codec Kontrollers
audio_description = ol.ip_dict['audio_codec_ctrl_0']
# Übergabe der Bezeichnung an Treiber
pAudio = a = AudioADAU1761(audio_description)
# Eintellen des Audiotreibers
pAudio.configure(sample_rate=48000, iic_index=1, uio_name='audio-codec-ctrl')

In [5]:
# einstellen der Lautsärke
pAudio.set_volume(60)

# Einstellen eingang: LineIn
pAudio.select_line_in()

# Einstellen eingang: HP/MIC
# pAudio.select_microphone()

In [26]:
# Echtzeitausgabe
Time = 20
pAudio.bypass(seconds=Time)

In [6]:
# Aufnahme
recTime = 10
pAudio.record(recTime)

In [7]:
# in pAudio.buffer werden die aufnahmen gespreichert
print(pAudio.buffer, type(pAudio.buffer))

[16777215 16777215 16777215 ... 16777215 16777215 16777215] <class 'numpy.ndarray'>


In [8]:
# Buffer ausspielen
pAudio.play()

In [16]:
# Echtzeitausgabe
Time = 5
pAudio.bypass(seconds=Time)